***
# Data Wrangling

I cleanned and reorganized the raw data collected from Glassdoor.com for data science purpose. To prepare for model building, I list the data wrangling plan below:

- Salary parsing
    
    The "Salary Estimate" was retrived as object (e.g. \$78K-\$133K (Glassdoor est.)). I removed     "$", "K", "-" and "(Glassdoor est.)" and only left numerical values. The salary                information will be represented by "max_salary", "min_salary" and "avg_salary". 

- Company name parsing

    The "Company" was collected as "_company name  company rating_" format (e.g. Amazon 4.0).     I removed the rating element from the column which makes the Company name text-only.

- Location parsing

    The "Location" column was collcted as "_city name, state abbreviation_" format (e.g.    Chicago, IL). For data science purpose, I decided to only use state information and removed the city information. Including the city information in the models would potentially decrease the efficiency. Using only state information is much more reasonable and effective.

- Age of Company

    The "Founded" column has information about when the company was founded. Instead of using the specific year, I transformed that information into the age of the company.

- Job description parsing

    The "Job description" column contains text information. However, it was very long. Since the goal of this project is to estimate salary, I only extracted useful information from the column. According to _"14 most used data science tools for 2019" (https://data-flair.training/blogs/data-science-tools/)_, python, r studio, spark, aws, excel, sas, matlab, tableau, tensorflow are widely used by data scientists. Thus, I am interested in how many companies would include those tools in their job description pages and what the correlation between having experiences with these tools and potentially earning a higher salary.

    

### Import packages

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data-scientist-salary-data.csv")
# Check NULL values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          1000 non-null   object 
 1   Salary Estimate    1000 non-null   object 
 2   Job Description    1000 non-null   object 
 3   Rating             1000 non-null   float64
 4   Company Name       1000 non-null   object 
 5   Location           1000 non-null   object 
 6   Headquarters       1000 non-null   int64  
 7   Size               1000 non-null   object 
 8   Founded            1000 non-null   int64  
 9   Type of ownership  1000 non-null   object 
 10  Industry           1000 non-null   object 
 11  Sector             1000 non-null   object 
 12  Revenue            1000 non-null   object 
 13  Competitors        1000 non-null   int64  
dtypes: float64(1), int64(3), object(10)
memory usage: 109.5+ KB


### Salary parsing

The "Salary Estimate" was retrived as object (e.g. \$78K-\$133K (Glassdoor est.)). I removed  "$", "K", "-" and "(Glassdoor est.)" and only left numerical values. The salary information    will be represented by "max_salary", "min_salary" and "avg_salary".

In [4]:
# remove "$", "K", "-" and "(Glassdoor est.)" and only left numerical values.
salary = df["Salary Estimate"].apply(lambda x: x.split("(")[0])
salary_minusdollorandk = salary.apply(lambda x: x.replace("K", "").replace("$", ""))

In [5]:
# create "min_salary" and "max_salary"
df["min_salary"] = salary_minusdollorandk.apply(lambda x: x.split("-")[0])
df["max_salary"] = salary_minusdollorandk.apply(lambda x: x.split("-")[1])

In [6]:
# convert into int64 format
df = df.astype({
    "min_salary":"int64",
    "max_salary":"int64"
})

In [7]:
# create "avg_salary"
df["avg_salary"] = (df["min_salary"] + df["max_salary"])/2

### Company name parsing

The "Company" was collected as "company name company rating" format (e.g. Amazon 4.0). I removed the rating element from the column which makes the Company name text-only.

In [8]:
# remove rating from company name column
df["company_text"] = df.apply(lambda x: x["Company Name"] if x["Rating"] < 0 else x["Company Name"][:-4], axis= 1)

### Location parsing

The "Location" column was collcted as "city name, state abbreviation" format (e.g. Chicago, IL). For data science purpose, I decided to only use state information and removed the city information. Including the city information in the models would potentially decrease the efficiency. Using only state information is much more reasonable and effective.

In [9]:
# remove city info from the column
df["job_state"] = df["Location"].apply(lambda x: x.split(",")[1].strip() if "," in x else x)
# Some location cells are extracted in different formats (e.g. Virginia, United States...)

In [10]:
# replace those values with standard state abbreviation
df["job_state"].replace({
    "United States":"US",
    "Virginia":"VA",
    "Massachusetts":"MA",
    "Utah":"UT",
    "New Jersey":"NJ",
    "Maryland":"MD",
    "Ohio":"OH",
    "California":"CA"
}, inplace= True)

In [11]:
# The "Headquarters" column is "-1" for all jobs. I guess Glassdoor.com made some changes and the scraper did not gather the information. So, I decided to drop the column
df.drop("Headquarters", axis= 1, inplace= True)

### Age of Company

The "Founded" column has information about when the company was founded. Instead of using the specific year, I transformed that information into the age of the company.

In [12]:
df["age"] = df["Founded"].apply(lambda x: x if x < 0 else 2020 - x)

In [20]:
# -1 means missing value, and we can see 143 companies did not report such information
df["age"][df["age"]== -1].count()

143

### Job description parsing

The "Job description" column contains text information. However, it was very long. Since the goal of this project is to estimate salary, I only extracted useful information from the column. According to "14 most used data science tools for 2019" (https://data-flair.training/blogs/data-science-tools/), python, r studio, spark, aws, excel, sas, matlab, tableau, tensorflow are widely used by data scientists. Thus, I am interested in how many companies would include those tools in their job description pages and what the correlation between having experiences with these tools and potentially earning a higher salary.

In [24]:
# create dummy variables which indicate whether a certain tool appeared in the job description

df["python_y/n"] = df["Job Description"].apply(lambda x: 1 if "python" in x.lower() else 0)
df["r_y/n"] = df["Job Description"].apply(lambda x: 1 if "r studio" in x.lower() or "r-studio" in x.lower() else 0)
df["spark_y/n"] = df["Job Description"].apply(lambda x: 1 if "spark" in x.lower() else 0)
df["aws_y/n"] = df["Job Description"].apply(lambda x: 1 if "aws" in x.lower() else 0)
df["excel_y/n"] = df["Job Description"].apply(lambda x: 1 if "excel" in x.lower() else 0)
df["sas_y/n"] = df["Job Description"].apply(lambda x: 1 if "sas" in x.lower() else 0)
df["matlab_y/n"] = df["Job Description"].apply(lambda x: 1 if "matlab" in x.lower() else 0)
df["tableau_y/n"] = df["Job Description"].apply(lambda x: 1 if "tableau" in x.lower() else 0)
df["tensorflow_y/n"] = df["Job Description"].apply(lambda x: 1 if "tensorflow" in x.lower() else 0)

### Export the cleaned csv

In [17]:
df.to_csv("data-scientist-salary-cleaned.csv", index= False)